In [46]:
import requests
import json
import pandas as pd
from getpass import getpass
import os
import time
from pymongo import MongoClient
from bson.decimal128 import Decimal128
from collections import defaultdict
from dotenv import load_dotenv
load_dotenv()

True

**MONGO  CONNECTION**

In [3]:
client = MongoClient("localhost:27017")
db = client["ironhack"]
c = db.get_collection("companies")

**FILTERING DATABASE WITH EMPLOYEES CONDITIONS**

In [26]:
def find_design_companies(collection):
    projection = {"_id": 0, "total_money_raised": 1, "name": 1, "category_code": 1, "offices.city": 1}
    design_companies = list(c.find({"category_code": "design"}, projection = projection).sort("total_money_raised", -1))
    return design_companies

design_companies = find_design_companies(c)
design_companies

[{'name': '99designs',
  'category_code': 'design',
  'total_money_raised': '$35M',
  'offices': [{'city': 'San Francisco'},
   {'city': 'Collingwood'},
   {'city': 'Berlin'}]},
 {'name': 'Moonfruit',
  'category_code': 'design',
  'total_money_raised': '$2.25M',
  'offices': [{'city': 'London'}]},
 {'name': 'Kickstarter',
  'category_code': 'design',
  'total_money_raised': '$10M',
  'offices': [{'city': 'Brooklyn'}]},
 {'name': 'Graticle',
  'category_code': 'design',
  'total_money_raised': '$0',
  'offices': [{'city': 'Ellensburg'}]}]

There are only 4 design companies so I won't be basing my decision on that, but I'll search some place where they can maybe go to conferencies or deign workshops. 

In [55]:
def find_successful_startups(collection):
    condition_1 = {"total_money_raised": {"$regex": "M"}}
    condition_2 = {"number_of_employees": {"$lte": 100}}
    projection = {"_id": 0, "total_money_raised": 1, "name": 1, "category_code": 1, "offices.city": 1}
    suc_startups = list(c.find({"$and": [condition_1,condition_2]}, projection = projection).sort("number_of_employees", -1))
    return suc_startups

successful_startups = find_successful_startups(c)
successful_startups

[{'name': 'Redfin',
  'category_code': 'real_estate',
  'total_money_raised': '$96.8M',
  'offices': [{'city': 'Seattle'}]},
 {'name': 'Simply Hired',
  'category_code': 'search',
  'total_money_raised': '$22.3M',
  'offices': [{'city': 'Sunnyvale'}]},
 {'name': 'hi5',
  'category_code': 'games_video',
  'total_money_raised': '$52M',
  'offices': [{'city': 'San Francisco'}]},
 {'name': 'mig33',
  'category_code': 'games_video',
  'total_money_raised': '$32.4M',
  'offices': [{'city': 'Singapore'}]},
 {'name': 'Fon',
  'category_code': 'mobile',
  'total_money_raised': '€44M',
  'offices': [{'city': 'Madrid'}]},
 {'name': 'EchoSign',
  'category_code': 'web',
  'total_money_raised': '$8.5M',
  'offices': [{'city': 'Palo Alto'}]},
 {'name': 'TrialPay',
  'category_code': 'advertising',
  'total_money_raised': '$55.8M',
  'offices': [{'city': 'Mountain View'}]},
 {'name': 'Firetide',
  'category_code': 'mobile',
  'total_money_raised': '$39.8M',
  'offices': [{'city': 'Campbell'}]},
 {'na

**EXPLORING DATABASE TO NARROW DOWN LOCATIONS**

From the successful startups database I'm going to find out in which cities they have offices. And for each city how manny offices in total. 

In [67]:
def oc_city_counts(collection,arg, arg2):

    city_counts = defaultdict(int)

    for item in collection:
        for office in item[arg]:
            city = office.get(arg2, '')
            if city:  
                city_counts[city] += 1

    sorted_city_counts = {k: v for k, v in sorted(city_counts.items(), key=lambda item: item[1], reverse=True)}

    cc = cc = pd.DataFrame(sorted_city_counts.values(), index=sorted_city_counts.keys(), columns=['Count'])
    return cc

city_counts = oc_city_counts(successful_startups, "offices", "city")
city_counts.head(50)


,Count
San Francisco,190
New York,127
London,78
Seattle,43
Palo Alto,41
San Mateo,38
Cambridge,31
Mountain View,27
Austin,27
Paris,25


In [68]:
city_counts = oc_city_counts(design_companies, "offices", "city")
city_counts


,Count
San Francisco,1
Collingwood,1
Berlin,1
London,1
Brooklyn,1
Ellensburg,1


EXPLAIN LO QUE FOUND OUT 3 CITIES
then mapa color cubre ciudad